In [1]:
import numpy as np
image_size = 28 # width and length
no_of_different_labels = 10 #  i.e. 0, 1, 2, 3, ..., 9
image_pixels = image_size * image_size
data_path = "data/mnist/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
                        delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
                       delimiter=",") 

截取数据并对数据进行预处理

In [2]:
train_data = train_data[:8000]
test_data = test_data[:2000]

In [6]:
fac = 0.99 / 255
train_imgs = np.asfarray(train_data[:, 1:]) * fac + 0.01
test_imgs = np.asfarray(test_data[:, 1:]) * fac + 0.01

train_labels = np.asfarray(train_data[:, :1])
test_labels = np.asfarray(test_data[:, :1])



lr = np.arange(no_of_different_labels)

# transform labels into one hot representation
train_labels_one_hot = (lr==train_labels).astype(float)
test_labels_one_hot = (lr==test_labels).astype(float)

# we don't want zeroes and ones in the labels neither:
train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import conv_nn

In [8]:
k=0
agent_number = 8
#初始化agent
ann_agent=[]
train_imgs_set=[]
train_labels_one_hot_set = []


for i in range(agent_number):
    ann_agent.append(conv_nn.ConvNeuralNetwork(no_of_in_nodes = image_pixels, 
                    no_of_out_nodes = 10, 
                    no_of_hidden_nodes = 100,
                    learning_rate = 0.1))
    train_imgs_set.append(train_imgs[1000*i : 1000*(i+1)])
    train_labels_one_hot_set.append(train_labels_one_hot[1000*i : 1000*(i+1)])
    
#初始化参数z，v_set

z, v = ann_agent[0].v_z_init_zero()

v_set = []
for i in range(agent_number):
    v_set.append(v)


In [9]:
#增广项的参数
c = 0.001
v_set[i][0].shape
[1,2] + [1,2]
len(train_imgs_set[i])

1000

In [10]:
#迭代次数
maxit = 1

while(k<maxit):
    #训练agent更新weight
    for i in range(agent_number):
        #每个agent
        for j in range(len(train_imgs_set[i])):
            ann_agent[i].train_ture_admm(train_imgs_set[i][j], train_labels_one_hot_set[i][j], c, z, v_set[i]) #为了方便

        corrects, wrongs = ann_agent[i].evaluate(train_imgs, train_labels)
        print("迭代：", k, "agent", i,"accuracy train: ", corrects / ( corrects + wrongs))
        corrects, wrongs = ann_agent[i].evaluate(test_imgs, test_labels)
        print("迭代：", k, "agent", i,"accuracy: test", corrects / ( corrects + wrongs))

    #更新参数z
    #初始化temp
    temp = []  
    for i in range(len(z)):
        temp.append(np.zeros(z[i].shape))
    for i in range(agent_number):
        temp[0] += (ann_agent[i].wih + 1.0 / c*v_set[i][0]) #这个的维数是
        temp[1] += (ann_agent[i].who + 1.0 / c*v_set[i][1])
    temp[0] = temp[0] / agent_number
    temp[1] = temp[1] / agent_number 
    
    z = temp 
    
    #c更新v_set
    for i in range(agent_number):
        v_set[i][0] += c*(ann_agent[i].wih - z[0])
        v_set[i][1] += c*(ann_agent[i].who - z[1])
    
    k += 1

迭代： 0 agent 0 accuracy train:  0.1385
迭代： 0 agent 0 accuracy: test 0.14
迭代： 0 agent 1 accuracy train:  0.150625
迭代： 0 agent 1 accuracy: test 0.179
迭代： 0 agent 2 accuracy train:  0.240375
迭代： 0 agent 2 accuracy: test 0.2575
迭代： 0 agent 3 accuracy train:  0.1365
迭代： 0 agent 3 accuracy: test 0.165
迭代： 0 agent 4 accuracy train:  0.0985
迭代： 0 agent 4 accuracy: test 0.1095
迭代： 0 agent 5 accuracy train:  0.0985
迭代： 0 agent 5 accuracy: test 0.1095
迭代： 0 agent 6 accuracy train:  0.172125
迭代： 0 agent 6 accuracy: test 0.164
迭代： 0 agent 7 accuracy train:  0.191625
迭代： 0 agent 7 accuracy: test 0.1825


In [11]:
numb = 1
corrects, wrongs = ann_agent[numb].evaluate(train_imgs, train_labels)
print("accuracy train: ", corrects / ( corrects + wrongs))
corrects, wrongs = ann_agent[numb].evaluate(test_imgs, test_labels)
print("accuracy: test", corrects / ( corrects + wrongs))

cm = ann_agent[numb].confusion_matrix(train_imgs, train_labels)
print(cm)

for i in range(10):
    print("digit: ", i, "precision: ", 
          ann_agent[numb].precision(i, cm), "recall: ", ann_agent[numb].recall(i, cm))

accuracy train:  0.150625
accuracy: test 0.179
[[ 39   2  11  68   6  51  11  87  25  14]
 [ 86   1   9   4   3   3   1  52   2   4]
 [116 410 283 311  21 127 160 170 216  95]
 [  3  19  26  15   5  17  17  11  54  18]
 [259 373 353 202 732 253 405 305 319 532]
 [ 32  26  24  20   5  42  77  33  21  40]
 [ 27  27  26  90   5  75   0  33  21  30]
 [ 23  12   1   2   0  12   5   0   1   1]
 [159  11  33  63  14  71  43  88  72  40]
 [ 51  25  22  36   8  51  79  74  22  21]]
digit:  0 precision:  0.04905660377358491 recall:  0.12420382165605096
digit:  1 precision:  0.0011037527593818985 recall:  0.006060606060606061
digit:  2 precision:  0.35913705583756345 recall:  0.14824515453116816
digit:  3 precision:  0.018495684340320593 recall:  0.08108108108108109
digit:  4 precision:  0.9161451814768461 recall:  0.1960889365121886
digit:  5 precision:  0.05982905982905983 recall:  0.13125
digit:  6 precision:  0.0 recall:  0.0
digit:  7 precision:  0.0 recall:  0.0
digit:  8 precision:  0.0956

In [12]:
i = 1
j = i + 1
print("wih_error: \n",ann_agent[i].wih - ann_agent[j].wih)
print("who_error: \n",ann_agent[i].who - ann_agent[j].who)

wih_error: 
 [[-1.83779126 -1.57540572 -1.14388605 ...  0.59049003  0.80278242
  -2.44075382]
 [-1.15225372  0.18692066 -0.2537315  ... -0.15579845  0.47012204
  -0.5965575 ]
 [-1.33952379  1.62279897  1.37816325 ... -1.66324404 -0.83021654
  -1.84549274]
 ...
 [ 0.26320698  1.8262967   0.77027012 ... -1.19772311  0.36856296
  -0.46006211]
 [-1.90990123 -1.8634495   0.44389099 ...  0.866382    2.13728801
  -0.53008712]
 [-1.46669807  0.19459381 -0.50137661 ... -0.57181105  0.30948248
  -0.41226317]]
who_error: 
 [[-1.14508997e+00 -9.31715622e-01  2.71336898e-01 -1.09667304e-01
  -8.10435888e-01 -1.69863517e+00  9.04053424e-02 -1.07521123e+00
   1.62427219e+00  1.05493132e+00 -8.49792295e-02 -1.30572756e+00
  -3.39074375e-01  1.92791261e+00  1.00623178e+00 -4.50221229e-01
   5.96552888e-01  4.07394694e-01  2.02470790e+00 -1.47741121e+00
  -1.79481166e+00 -6.07595779e-01 -1.33687915e+00  5.57923458e-01
  -2.03195913e+00  9.95441196e-01 -8.74168112e-01 -7.04607767e-01
  -6.74497356e-02 -1